In [56]:
import numpy as np 
import pandas as pd

In [57]:
df = pd.read_csv('data.csv')
df = df[['IfAcceptService','LessThan500','500To1000','1000To1500','NoLessThan1500','IfShoeLover']]
for col_name in ['IfAcceptService','LessThan500','500To1000','1000To1500','NoLessThan1500','IfShoeLover']:
    if col_name == 'IfShoeLover':
        for row_index in range(df.shape[0]):
            if df.loc[row_index,col_name] == 2:
                df.loc[row_index,col_name] = 0
        continue
    if col_name == 'IfAcceptService':
        for row_index in range(df.shape[0]):
            if df.loc[row_index,col_name] == 2:
                df.loc[row_index,col_name] = 0
        continue

    for row_index in range(df.shape[0]):
        if df.loc[row_index,col_name] == 1:
            df.loc[row_index,col_name] = 0
        else:
            df.loc[row_index,col_name] = 1

data = df[['IfAcceptService','LessThan500','500To1000','1000To1500','NoLessThan1500']].values
target = df['IfShoeLover'].values

train_set,test_set = df[0:150],df[150:].reset_index()[['IfAcceptService','LessThan500','500To1000','1000To1500','NoLessThan1500','IfShoeLover']]

train_set_data = df.loc[0:149,['IfAcceptService','LessThan500','500To1000','1000To1500','NoLessThan1500']]
train_set_target = df.loc[0:149,['IfShoeLover']]

test_set_data = df.loc[150:,['IfAcceptService','LessThan500','500To1000','1000To1500','NoLessThan1500']].reset_index()[['IfAcceptService','LessThan500','500To1000','1000To1500','NoLessThan1500']]
test_set_target = df.loc[150:,['IfShoeLover']].reset_index()['IfShoeLover']

In [58]:
# KNN直接调包
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=2)
classifier.fit(train_set_data,train_set_target)
classifier.score(test_set_data,test_set_target)

0.6530612244897959

In [59]:
# 手写KNN
# vote
def knn_vote(best_k,train_df,columns):
      
    vote_list = train_df.loc[best_k,columns[-2]]
    mode = vote_list.mode().get(0)
    return mode

# calculate distance
def cal_dis(vec_train,vec_test):
    
    vec_train,vec_test = np.array(vec_train), np.array(vec_test)
    dis = np.sqrt(np.sum(np.square(vec_train - vec_test)))
    return dis

def cal_correct_rate(test_df,columns):
    
    # 构建混淆矩阵
    labels = list(test_df[columns[-2]].unique())
    mat_shape = len(labels)
    conf_mat = np.zeros([mat_shape,mat_shape])
    
    correct_sum = 0
    for i in range(test_df.shape[0]):
        true_label_idx = labels.index(test_df.at[i,columns[-2]])
        pred_label_idx = labels.index(test_df.at[i,columns[-1]])
        conf_mat[true_label_idx][pred_label_idx] += 1
        if test_df.at[i,columns[-2]] == test_df.at[i,columns[-1]]:
            correct_sum += 1
    return conf_mat,correct_sum / test_df.shape[0]

# knn main function
def knn_classify_main(train_df,test_df,columns,k=2):
    
    # add a new column to record classification result
    columns.append('class_classify')
    test_df.insert(len(columns)-1,'class_classify','')
    
    for i in range(test_df.shape[0]):
        # get test_data vector
        vec_test = []
        for ii in range(len(columns) -2):
            vec_test.append(test_df.iat[i,ii])
        
        best_k , best_k_dis = [], []
        for j in range(train_df.shape[0]):
            # get train_data vector
            vec_train = []
            for jj in range(len(columns) -2):
                vec_train.append(train_df.iat[j,jj])
            # calculate distance
            dis = cal_dis(vec_train,vec_test)
            if j < k: # add k initial elems
                best_k.append(j)
                best_k_dis.append(dis)
            else: # substitute elem
                for kk in range(len(best_k)):
                    if dis < best_k_dis[kk]:
                        best_k[kk] = j
                        best_k_dis[kk] = dis
                        break   
        
        vote_res = knn_vote(best_k,train_df,columns)
        test_df.at[i,columns[-1]] = vote_res
       
    conf_mat,correct_rate = cal_correct_rate(test_df,columns)
    return test_df,conf_mat,correct_rate
columns = ['LessThan500','500To1000','1000To1500','NoLessThan1500','IfShoeLover']
train_data,test_data = train_set,test_set
test_df,conf_mat,correct_rate = knn_classify_main(train_data,test_data,columns,k=2)
print(correct_rate)
print(conf_mat)

0.673469387755102
[[ 4. 12.]
 [ 4. 29.]]


In [61]:
import torch
import torch.nn as nn 
import torchvision 
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader

from sklearn.metrics import accuracy_score

device = torch.device('cpu')

# 超参数
input_size = 5 # 一共五个特征
hidden_size = 100 # 隐层神经元个数
num_classes = 2 # 类别总数
num_epochs = 10000 # 训练轮数
#batch_size = 15 # 批处理大小
learning_rate = 0.001

# 全连接神经网路
class FCNetWork(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(FCNetWork,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,num_classes)
    
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out 
    
    def predict(self,x):
        pred = F.softmax(self.forward(x))
        ans = []
        for t in pred:
            if t[0] > t[1]:
                ans.append(0)
            else:
                ans.append(1)
        return torch.tensor(ans)

# class My_dataset(Dataset):
#     def __init__(self,data,target):
#         super().__init__()
#         self.src = torch.tensor(data.values).type(torch.FloatTensor)
#         self.trg = torch.tensor(target.values).type(LongTensor)
    
#     def __getitem__(self,index):
#         return self.src[index],self.trg[index]
    
#     def __len__(self):
#         return len(self.src)


model = FCNetWork(input_size,hidden_size,num_classes).to(device)

# 确定损失函数和优化器
criterion = nn.CrossEntropyLoss() # 交叉熵
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

X = torch.tensor(train_set_data.values).type(torch.FloatTensor)
y = torch.tensor(train_set_target.values).type(torch.LongTensor)

losses = []
for i in range(num_epochs):
    y_pred = model.forward(X)
    loss = criterion(y_pred,y.squeeze())
    if i % 10 == 0:
        print('Loss is {}'.format(loss.item()))
    losses.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# train_data = My_dataset(train_set_data,train_set_target)
# test_data = My_dataset(test_set_data,test_set_target)

# train_loader = torch.utils.data.DataLoader(
#     dataset=train_data,
#     batch_size=batch_size,
#     shuffle=True
# )

# test_loader = torch.utils.data.DataLoader(
#     dataset=test_data,
#     batch_size=int(batch_size/3),
#     shuffle=False
# )
# 开始训练模型
# total_step = len(train_loader)
# for epoch in range(num_epochs):
#     for i,(data,target) in enumerate(train_loader):
#         data = data.to(device)
#         target = target.to(device)

#         # 向前传播
#         out = model(data)
#         loss = loss_func(out,target.squeeze())

#         # 反向传播
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         if (i+1) % 5 == 0:
#             print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
#                    .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

45228385925
Loss is 0.2131698578596115
Loss is 0.21316847205162048
Loss is 0.213166743516922
Loss is 0.213165283203125
Loss is 0.21316386759281158
Loss is 0.21316233277320862
Loss is 0.21316099166870117
Loss is 0.21315963566303253
Loss is 0.21315820515155792
Loss is 0.21315675973892212
Loss is 0.2131555676460266
Loss is 0.21315419673919678
Loss is 0.21315275132656097
Loss is 0.21315127611160278
Loss is 0.21315020322799683
Loss is 0.2131485491991043
Loss is 0.21314747631549835
Loss is 0.2131463587284088
Loss is 0.2131449431180954
Loss is 0.2131435126066208
Loss is 0.213142529129982
Loss is 0.2131413221359253
Loss is 0.21314001083374023
Loss is 0.21313871443271637
Loss is 0.21313773095607758
Loss is 0.2131366729736328
Loss is 0.21313533186912537
Loss is 0.21313416957855225
Loss is 0.21313299238681793
Loss is 0.21313200891017914
Loss is 0.21313075721263885
Loss is 0.21312952041625977
Loss is 0.21312858164310455
Loss is 0.21312746405601501
Loss is 0.21312640607357025
Loss is 0.213125362992

In [ ]:
print(accuracy_score(model.predict(X),y))

In [63]:
X = torch.tensor(test_set_data.values).type(torch.FloatTensor)
y = torch.tensor(test_set_target.values).type(torch.LongTensor)
print(accuracy_score(model.predict(X),y))

0.6938775510204082
